In [5]:
import pandas as pd
import numpy as np
import cvxpy as cp

### Load data

In [6]:
data = pd.read_csv(r'Course_Instructor.csv') # the last two boolean var. columns are False even when afctually missing

# Drop rows with missing values in the first 8 columns
data = data.dropna(subset=data.columns[:8], how='all')

print(data.head())

  Course Name  Course Section    Days   Start Time     End Time  \
0   MATH 2568         27178.0  MoWeFr   8:00:00 AM   8:55:00 AM   
1   MATH 2568         27181.0  MoWeFr  11:30:00 AM  12:25:00 PM   
2   MATH 2568         27182.0  MoWeFr  12:40:00 PM   1:35:00 PM   
3   MATH 2568         27183.0  MoWeFr   1:50:00 PM   2:45:00 PM   
4   MATH 2568         27184.0  MoWeFr  12:40:00 PM   1:35:00 PM   

              Instructor  Instructor Rating  Unit Is GE 2nd Level Writing?  \
0  Esteban Vargas Bernal                4.6   3.0                    False   
1             Liding Yao                1.5   3.0                    False   
2             Liding Yao                1.5   3.0                    False   
3            Luis Casian                2.9   3.0                    False   
4        Arman Darbinyan                1.0   3.0                    False   

  Is GE Cultures and Ideas?  
0                     False  
1                     False  
2                     False  
3       

In [7]:
nrow = len(data)
ncol = len(data.columns)
clean = pd.DataFrame(columns =('Course Name, Section', 'Start, End', 'isDay', 'Instructor', 'Ins Rating', 'Unit', 'Is 2nd Lvl W', 'Is CI'))
clean.head()

def time_to_integer(time_str):
    return int(time_str.replace(':', '')) # convert time to interger military time for convinience

for index, row in data.iterrows():
    combined_days = row['Days']
    start_time = time_to_integer(pd.to_datetime(row['Start Time']).strftime('%H:%M'))  # Convert to integer
    end_time = time_to_integer((pd.to_datetime(row['End Time']) + pd.Timedelta(minutes=15)).strftime('%H:%M'))  # Add 15 minutes and convert to integer

    is_monday = 'Mo' in combined_days
    is_tuesday = 'Tu' in combined_days
    is_wednesday = 'We' in combined_days
    is_thursday = 'Th' in combined_days
    is_friday = 'Fr' in combined_days

    clean.loc[len(clean.index)] = [(row['Course Name'], str(int(row['Course Section']))), (start_time, end_time), (is_monday, is_tuesday, is_wednesday, is_thursday, is_friday), row['Instructor'], row['Instructor Rating'], row['Unit'], row['Is GE 2nd Level Writing?'], row['Is GE Cultures and Ideas?']]

clean.fillna(0, inplace=True)
clean.head()

,"Course Name, Section","Start, End",isDay,Instructor,Ins Rating,Unit,Is 2nd Lvl W,Is CI
0,"(MATH 2568, 27178)","(800, 910)","(True, False, True, False, True)",Esteban Vargas Bernal,4.6,3.0,False,False
1,"(MATH 2568, 27181)","(1130, 1240)","(True, False, True, False, True)",Liding Yao,1.5,3.0,False,False
2,"(MATH 2568, 27182)","(1240, 1350)","(True, False, True, False, True)",Liding Yao,1.5,3.0,False,False
3,"(MATH 2568, 27183)","(1350, 1500)","(True, False, True, False, True)",Luis Casian,2.9,3.0,False,False
4,"(MATH 2568, 27184)","(1240, 1350)","(True, False, True, False, True)",Arman Darbinyan,1.0,3.0,False,False


### Process data and create helper function

In [8]:
course_sections = {}

for course, section in clean['Course Name, Section']:
    if course not in course_sections:
        course_sections[course] = []
    course_sections[course].append(section)

In [9]:
# Indices for each class
course_section_map = {(course, section): (i, j) for i, (course, sections) in enumerate(course_sections.items()) for j, section in enumerate(sections)}
get_course_section_map = {value: key for key, value in course_section_map.items()}
get_course_section_map

{(0, 0): ('MATH 2568', '27178'),
 (0, 1): ('MATH 2568', '27181'),
 (0, 2): ('MATH 2568', '27182'),
 (0, 3): ('MATH 2568', '27183'),
 (0, 4): ('MATH 2568', '27184'),
 (0, 5): ('MATH 2568', '27185'),
 (1, 0): ('STAT 3202', '27866'),
 (1, 1): ('STAT 3202', '27902'),
 (1, 2): ('STAT 3202', '20935'),
 (1, 3): ('STAT 3202', '29400'),
 (1, 4): ('STAT 3202', '36981'),
 (1, 5): ('STAT 3202', '27253'),
 (1, 6): ('STAT 3202', '27254'),
 (1, 7): ('STAT 3202', '27256'),
 (2, 0): ('CSE 2421', '6973'),
 (2, 1): ('CSE 2421', '7009'),
 (2, 2): ('CSE 2421', '7085'),
 (2, 3): ('CSE 2421', '7239'),
 (2, 4): ('CSE 2421', '7327'),
 (2, 5): ('CSE 2421', '11915'),
 (2, 6): ('CSE 2421', '12387'),
 (2, 7): ('CSE 2421', '36975'),
 (3, 0): ('CSE 3430', '7104'),
 (3, 1): ('CSE 3430', '7333'),
 (4, 0): ('ARTEDUC 2700', '17849'),
 (4, 1): ('ARTEDUC 2700', '17850'),
 (4, 2): ('ARTEDUC 2700', '17852'),
 (5, 0): ('PHR 2367.01', '17021'),
 (5, 1): ('PHR 2367.01', '17168'),
 (6, 0): ('ENGR 2367', '8154'),
 (6, 1): ('ENGR

In [10]:
num_courses = len(course_sections)
max_sections = max(len(sections) for sections in course_sections.values())

# X_{i, j} - i = # course; j = # section
X = cp.Variable((num_courses, max_sections), boolean=True)

In [11]:
instructor_ratings = np.zeros((num_courses, max_sections))
course_units = np.zeros((num_courses, max_sections))

for i in range(num_courses):
    for j in range(max_sections):
        course_selection = get_course_section_map.get((i, j))
        if course_selection is not None:
            row = clean.loc[clean["Course Name, Section"] == course_selection]
            rating = row['Ins Rating']
            unit = row['Unit']
            if rating is not None:
                instructor_ratings[i, j] = rating
            if unit is not None:
                course_units[i, j] = unit


In [12]:
which_2ndlvl = []
writings = clean.loc[clean['Is 2nd Lvl W']==True]['Course Name, Section'].tolist()
for writing in writings:
    which_2ndlvl.append(course_section_map[writing])

which_CI = []
CIs = clean.loc[clean['Is CI']==True]['Course Name, Section'].tolist()
for CI in CIs:
    which_CI.append(course_section_map[CI])

which_s3202_lec = []
stat_3202 = clean[clean['Course Name, Section'].apply(lambda x: x[0]) == 'STAT 3202']
s3202_lecs = stat_3202[stat_3202['Unit']==4]['Course Name, Section']
for s3202_lec in s3202_lecs:
    which_s3202_lec.append(course_section_map[s3202_lec])

which_s3202_rec = []
s3202_recs = stat_3202[stat_3202['Unit']==0]['Course Name, Section']
for s3202_rec in s3202_recs:
    which_s3202_rec.append(course_section_map[s3202_rec])

In [13]:
def check_timing_clash(class_a_timing, class_b_timing):
    start_a, end_a = class_a_timing
    start_b, end_b = class_b_timing


    # Check if there's a clash between class_a and class_b timings
    if start_a < end_b and start_b < end_a:
        return True  # There's a clash
    else:
        return False  # No clash

### Optimization programming

In [14]:
# Objective: Indicator * Ratings
obj_func = cp.sum(cp.multiply(X, instructor_ratings))

In [15]:
# Constraints
constraints = []

In [16]:
# Ignore all variables not in the course_section map (If not in the list, set equal to zero)
for i in range(num_courses):
    for j in range(max_sections):
        if (i, j) not in set(val for val in course_section_map.values()):
            constraints.append(X[i, j] == 0)

# Constraints on credit hour
constraints.append(cp.sum([X[i, j] for i, j in which_2ndlvl]) == 1) # one 2nd Lvl Writing course required
constraints.append(cp.sum([X[i, j] for i, j in which_CI]) == 1) # one Cult. & Ideas courses required
constraints.append(cp.sum(X[0,:]) == 1) # one M2568 required
constraints.append(cp.sum(X[1,:]) == 2) # one S3202 (lec + rec) required
constraints.append(cp.sum(X[2,:]+X[3,:]) == 1) # either one C2421 or C3430 required
# one lecture and one recitation for stat3202
constraints.append(cp.sum([X[i, j] for i, j in which_s3202_rec]) + cp.sum([X[i, j] for i, j in which_s3202_lec]) == 2)

# min and max credit hour limits for a full time student
constraints.append(cp.sum(cp.multiply(X, course_units)) <= 18)
constraints.append(cp.sum(cp.multiply(X, course_units)) >= 12)


In [17]:
# Add constraints to ensure potential timing clashes between classes on shared days
for i, class_a in clean.iterrows():
    for j, class_b in clean.iterrows():
        if i != j:  # Ensure different classes
            common_days = set(class_a['isDay']) & set(class_b['isDay'])  # Find common days
            if common_days:  # If classes share at least one common day
                for day in common_days:
                    if check_timing_clash(class_a['Start, End'], class_b['Start, End']):
                        # Replace course_section_map[(class_a['Course Name, Section'])] with the respective index
                        constraints.append(X[course_section_map[class_a['Course Name, Section']]] +
                                           X[course_section_map[class_b['Course Name, Section']]] <= 1)

In [18]:
problem = cp.Problem(cp.Maximize(obj_func), constraints)

problem.solve(solver=cp.GUROBI, verbose = True)

print("obj_func =")
print(obj_func.value)
print("X =")
print(np.round(X.value))

                                     CVXPY                                     
                                    v1.1.18                                    
(CVXPY) Nov 29 06:02:31 PM: Your problem has 80 variables, 603 constraints, and 0 parameters.
(CVXPY) Nov 29 06:02:31 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 29 06:02:31 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 29 06:02:31 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 29 06:02:31 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Nov 29 06:02:31 PM: Reduction chain: FlipObjective -> CvxAttr2Constr -> Qp2SymbolicQ

### Result

In [19]:
course_indices = np.where(X.value == 1)

In [39]:
course_indices = np.where(X.value == 1)

course_selection = []
for i in range(len(course_indices[0])):
    course_selection.append(get_course_section_map.get((course_indices[0][i], course_indices[1][i])))
    
section_selection = [item[1] for item in course_selection]
optimized_courses = data[data['Course Section'].isin(section_selection)]

In [40]:
optimized_courses

,Course Name,Course Section,Days,Start Time,End Time,Instructor,Instructor Rating,Unit,Is GE 2nd Level Writing?,Is GE Cultures and Ideas?
0,MATH 2568,27178.0,MoWeFr,8:00:00 AM,8:55:00 AM,Esteban Vargas Bernal,4.6,3.0,False,False
6,STAT 3202,27866.0,TuTh,9:35:00 AM,10:55:00 AM,Sanjeewani Weerasingha,3.2,4.0,False,False
9,STAT 3202,29400.0,Mo,12:40:00 PM,1:35:00 PM,Alan Gan,NaN,0.0,False,False
21,CSE 2421,36975.0,TuWeThFr,11:30:00 AM,12:25:00 PM,Neil Kirby,4.1,4.0,False,NaN
30,ENGR 2367,8154.0,TuTh,3:55:00 PM,5:15:00 PM,Angie Burks,5.0,3.0,True,False
38,ANTHR 1100,20.0,TuTh,2:20:00 PM,3:40:00 PM,Jeffrey Peterson,5.0,3.0,False,True
